In [ ]:
# Imports
import sqlite3

In [ ]:
# Instantiates the connection and the cursor to the database.
connection = sqlite3.connect("mecenact.db")
cursor = connection.cursor()

In [ ]:
# Creates the table to store the users' information.
cursor.execute("""
            CREATE TABLE users (
            username TEXT,
            password TEXT NOT NULL,
            post_count INTEGER NOT NULL,
            PRIMARY KEY (username) )
            """)

In [ ]:
# Creates the table to store the posts' information.
cursor.execute("""
            CREATE TABLE posts (
            author TEXT,
            id INTEGER,
            title TEXT NOT NULL,
            image BLOB NOT NULL,
            body TEXT,
            PRIMARY KEY (id, author),
            FOREIGN KEY (author) 
               REFERENCES users (username)
               ON DELETE CASCADE )
            """)

In [ ]:
# Creates the table to store the comments' information.
cursor.execute("""
            CREATE TABLE tags (
            tag TEXT,
            author TEXT,
            post_id INTEGER,
            PRIMARY KEY (tag, author, post_id),
            FOREIGN KEY (author, post_id) 
               REFERENCES posts (author, id) 
               ON DELETE CASCADE )
            """)

In [ ]:
# Commit any changes and close the connection.
connection.commit()
connection.close()

In [ ]:
# Imports for random generation and model declaration.
# Generating random post titles.
from wonderwords import RandomSentence
sentence_model = RandomSentence()

# Generating random post bodies.
from transformers import pipeline
paragraph_model = pipeline('text-generation', model = 'gpt2')

# Generating random images.
import requests

# Generating random usernames.
import names

# Generating random passwords.
import random
import string

# Generating random tags.
from wonderwords import RandomWord
word_model = RandomWord()

In [ ]:
# Imports for database manipulation.
import db_api as db
db.db_path = 'mecenact.db'

In [ ]:
# Random user generation.
def random_user():
    username = names.get_full_name()
    password = ''.join(random.choice(string.ascii_letters + string.digits + string.punctuation) for i in range(random.randint(1, 16)))
    return username, password

# Random post generation and output formatting.
def random_post():
    sentence = sentence_model.sentence()

    paragraph = paragraph_model(sentence, do_sample = True, top_k = 50, temperature = 0.9, max_length = 100)[0]['generated_text']
    paragraph = paragraph.split('.')
    paragraph.pop(0)
    paragraph.pop()
    paragraph = '.'.join(paragraph) + '.'

    response = requests.get('https://source.unsplash.com/random')
    image = response.content

    return sentence, image, paragraph

# Random tag generation.
def random_tag():
    return word_model.word()

In [ ]:
# Parameters.
num_of_users = 20
num_of_posts = 100
num_of_tags = 50

# Generates random users and stores them in the database.
users = []
i = 0
while i < num_of_users:
    username, password = random_user()
    if username not in users:
        users.append(username)
        db.create_user(username, password)
        i = i + 1

# Generates random posts and stores them in the database.
posts = []
for i in range(num_of_posts):
    author = random.choice(users)
    title, image, body = random_post()
    posts.append(db.create_post(author, title, image, body))

# Generates random tags.
tags = []
i = 0
while i < num_of_tags:
    tag = random_tag()
    if tag not in tags: 
        tags.append(tag)
        i = i + 1

# Assigns random tags to each post.
for post in posts:
    post_tags = random.sample(tags, random.randint(1, 5))
    db.assign_tags(post_tags, post[0], post[1])